In [3]:
from collections import deque

# Define variables
people = ["George", "John", "Robert", "Barbara", "Christine", "Yolanda"]
rooms = ["Bathroom", "Dining Room", "Kitchen", "Living Room", "Pantry", "Study"]
weapons = ["Bag", "Firearm", "Gas", "Knife", "Poison", "Rope"]

# Set of all assignments
assignments = []
num_checks = 0

# Define domains
domains = {
    person: set(rooms) for person in people
}
domains.update({
    room: set(weapons) for room in rooms
})

# Clues as constraints
def is_valid(assignment):
    """
    Check if the current assignment satisfies all clues.
    """
    global num_checks
    num_checks += 1

    # Extract assignments for people, rooms, and weapons
    person_to_room = {person: room for person, room, weapon in assignment}
    person_to_weapon = {person: weapon for person, room, weapon in assignment}
    room_to_weapon = {room: weapon for person, room, weapon in assignment}

    # Clue 1: The man in the kitchen was not found with the rope, knife, or bag.
    if person_to_room.get("George", "") == "Kitchen" or \
       person_to_room.get("John", "") == "Kitchen" or \
       person_to_room.get("Robert", "") == "Kitchen":
        kitchen_weapon = room_to_weapon.get("Kitchen", "")
        if kitchen_weapon in ["Rope", "Knife", "Bag", "Firearm"]:
            return False
    
    # Clue 2: Barbara was either in the study or the bathroom; Yolanda was in the other.
    if person_to_room.get("Barbara", "") not in ["Study", "Bathroom", ""]:
        return False
    if person_to_room.get("Yolanda", "") not in ["Study", "Bathroom", ""]:
        return False
    
    # Clue 3: The person with the bag, who was not Barbara nor George, was not in the bathroom nor the dining room.
    for person, room, weapon in assignment:
        if weapon == "Bag" and (person == "Barbara" or person == "George"):
            return False
        if weapon == "Bag" and room in ["Bathroom", "Dining Room"]:
            return False

    # Clue 4: The woman with the rope was found in the study.
    for person, room, weapon in assignment:
        if weapon == "Rope" and room != "Study":
            return False

    # Clue 5: The weapon in the living room was found with either John or George.
    living_room_weapon = room_to_weapon.get("Living Room", "")
    if living_room_weapon and person_to_room.get("John", "") != "Living Room" and \
       person_to_room.get("George", "") != "Living Room":
        return False
    
    # Clue 6: The knife was not in the dining room.
    if room_to_weapon.get("Dining Room", "") == "Knife":
        return False

    # Clue 7: Yolanda was not with the weapon found in the study nor the pantry.
    study_weapon = room_to_weapon.get("Study", "")
    pantry_weapon = room_to_weapon.get("Pantry", "")
    if person_to_room.get("Yolanda", "") in ["Study", "Pantry"]:
        return False

    # Clue 8: The firearm was in the room with George.
    if person_to_weapon.get("George", "") not in ["Firearm", ""]:
        return False
    
    # Final clue: Mr. Boddy was gassed in the pantry, and the suspect found in that room is the murderer.
    if room_to_weapon.get("Pantry", "") not in ["Gas", ""]:
        return False
    
    return True

# Implementing the AC3 algorithm
def ac3(domains):
    queue = deque([(xi, xj) for xi in domains for xj in domains if xi != xj])

    while queue:
        (xi, xj) = queue.popleft()

        if revise(domains, xi, xj):
            if not domains[xi]:
                return False  # No solution exists
            for xk in (domains.keys() - {xi, xj}):
                queue.append((xk, xi))
    return True

def revise(domains, xi, xj):
    revised = False
    for x in set(domains[xi]):
        if not any(satisfies_constraints(xi, x, xj, y) for y in domains[xj]):
            domains[xi].remove(x)
            revised = True
    return revised

def satisfies_constraints(xi, vx, xj, vy):
    # Construct a temporary assignment with placeholder values to match the expected format
    if xi in people:
        temp_assignment = [(xi, vx, "PlaceholderWeapon")]
    elif xi in rooms:
        temp_assignment = [("PlaceholderPerson", xi, vx)]
    else:
        temp_assignment = [("PlaceholderPerson", "PlaceholderRoom", xi)]
    
    if xj in people:
        temp_assignment.append((xj, vy, "PlaceholderWeapon"))
    elif xj in rooms:
        temp_assignment.append(("PlaceholderPerson", xj, vy))
    else:
        temp_assignment.append(("PlaceholderPerson", "PlaceholderRoom", xj))
    
    return is_valid(temp_assignment)

# Preprocess with AC3 to enforce arc-consistency
if ac3(domains):
    print("AC3 succeeded, domains are arc-consistent.")
else:
    print("AC3 failed, no solution exists.")

# Backtracking algorithm
def backtrack(assignment):
    """
    Perform backtracking to solve the puzzle.
    """
    if len(assignment) == 6:
        return assignment
    
    # Try all rooms and weapons
    for person in people:
        if person not in [p for p, r, w in assignment]:
            for room in rooms:
                if room not in [r for p, r, w in assignment]:
                    for weapon in weapons:
                        if weapon not in [w for p, r, w in assignment]:
                            new_assignment = assignment + [(person, room, weapon)]
                            if is_valid(new_assignment):
                                result = backtrack(new_assignment)
                                if result:
                                    return result
    
    return None

# Find the solution
solution = backtrack(assignments)

# Print the solution
if solution:
    print("Solution found:")
    for person, room, weapon in solution:
        print(f"{person} was in the {room} with the {weapon}")
    print('Num Checks', num_checks)
else:
    print("No solution found.")

num_checks = 0

# Function to perform forward checking
def forward_check(domains, person, room, weapon):
    temp_domains = {k: v.copy() for k, v in domains.items()}
    # Remove the assigned values from the domains of other variables
    for other_person in people:
        if other_person != person:
            temp_domains[other_person].discard(room)
    for other_room in rooms:
        if other_room != room:
            temp_domains[other_room].discard(weapon)
    return temp_domains

# Backtracking algorithm with forward checking
def backtrack(assignment, domains):
    """
    Perform backtracking with forward checking to solve the puzzle.
    """
    if len(assignment) == 6:
        return assignment

    # Try all rooms and weapons
    for person in people:
        if person not in [p for p, r, w in assignment]:
            for room in rooms:
                if room not in [r for p, r, w in assignment]:
                    for weapon in weapons:
                        if weapon not in [w for p, r, w in assignment]:
                            new_assignment = assignment + [(person, room, weapon)]
                            if is_valid(new_assignment):
                                # Forward check
                                new_domains = forward_check(domains, person, room, weapon)
                                result = backtrack(new_assignment, new_domains)
                                if result:
                                    return result
    
    return None

# Find the solution
solution = backtrack(assignments, domains)

# Print the solution
if solution:
    print("Solution found:")
    for person, room, weapon in solution:
        print(f"{person} was in the {room} with the {weapon}")
    print('Num Checks', num_checks)
else:
    print("No solution found.")

AC3 failed, no solution exists.
Solution found:
George was in the Dining Room with the Firearm
John was in the Living Room with the Bag
Robert was in the Kitchen with the Poison
Barbara was in the Study with the Rope
Christine was in the Pantry with the Gas
Yolanda was in the Bathroom with the Knife
Num Checks 46894
Solution found:
George was in the Dining Room with the Firearm
John was in the Living Room with the Bag
Robert was in the Kitchen with the Poison
Barbara was in the Study with the Rope
Christine was in the Pantry with the Gas
Yolanda was in the Bathroom with the Knife
Num Checks 46858


In [ ]:
from collections import deque

# Define variables
people = ["George", "John", "Robert", "Barbara", "Christine", "Yolanda"]
rooms = ["Bathroom", "Dining Room", "Kitchen", "Living Room", "Pantry", "Study"]
weapons = ["Bag", "Firearm", "Gas", "Knife", "Poison", "Rope"]

# Set of all assignments
assignments = []
num_checks = 0

# Define domains
# Each person can be in any room
domains = {
    person: set(rooms) for person in people
}
# Each room can contain any weapon
domains.update({
    room: set(weapons) for room in rooms
})

# Clues as constraints
def is_valid(assignment):
    """
    Check if the current assignment satisfies all clues.
    """
    global num_checks
    num_checks += 1

    # Extract assignments for people, rooms, and weapons
    person_to_room = {person: room for person, room, weapon in assignment}
    person_to_weapon = {person: weapon for person, room, weapon in assignment}
    room_to_weapon = {room: weapon for person, room, weapon in assignment}

    # Clue 1: The man in the kitchen was not found with the rope, knife, or bag.
    if person_to_room.get("George", "") == "Kitchen" or \
       person_to_room.get("John", "") == "Kitchen" or \
       person_to_room.get("Robert", "") == "Kitchen":
        kitchen_weapon = room_to_weapon.get("Kitchen", "")
        if kitchen_weapon in ["Rope", "Knife", "Bag", "Firearm"]:
            return False
    
    # Clue 2: Barbara was either in the study or the bathroom; Yolanda was in the other.
    if person_to_room.get("Barbara", "") not in ["Study", "Bathroom", ""]:
        return False
    if person_to_room.get("Yolanda", "") not in ["Study", "Bathroom", ""]:
        return False
    
    # Clue 3: The person with the bag, who was not Barbara nor George, was not in the bathroom nor the dining room.
    for person, room, weapon in assignment:
        if weapon == "Bag" and (person == "Barbara" or person == "George"):
            return False
        if weapon == "Bag" and room in ["Bathroom", "Dining Room"]:
            return False

    # Clue 4: The woman with the rope was found in the study.
    for person, room, weapon in assignment:
        if weapon == "Rope" and room != "Study":
            return False

    # Clue 5: The weapon in the living room was found with either John or George.
    living_room_weapon = room_to_weapon.get("Living Room", "")
    if living_room_weapon and person_to_room.get("John", "") != "Living Room" and \
       person_to_room.get("George", "") != "Living Room":
        return False
    
    # Clue 6: The knife was not in the dining room.
    if room_to_weapon.get("Dining Room", "") == "Knife":
        return False

    # Clue 7: Yolanda was not with the weapon found in the study nor the pantry.
    study_weapon = room_to_weapon.get("Study", "")
    pantry_weapon = room_to_weapon.get("Pantry", "")
    if person_to_room.get("Yolanda", "") in ["Study", "Pantry"]:
        return False

    # Clue 8: The firearm was in the room with George.
    if person_to_weapon.get("George", "") not in ["Firearm", ""]:
        return False
    
    # Final clue: Mr. Boddy was gassed in the pantry, and the suspect found in that room is the murderer.
    if room_to_weapon.get("Pantry", "") not in ["Gas", ""]:
        return False
    
    return True

# Implementing the AC3 algorithm
def ac3(domains):
    """
    Arc-Consistency 3 (AC3) Algorithm
    ---------------------------------
    Ensures that the problem is arc-consistent by iteratively 
    examining pairs of variables (arcs) and pruning the domains of variables
    to remove values that cannot possibly be part of a solution.

    Parameters:
    - domains: A dictionary representing the domains of each variable.

    Returns:
    - True if the domains are arc-consistent, False otherwise.
    """
    # Initialize the queue with all arcs (pairs of variables)
    queue = deque([(xi, xj) for xi in domains for xj in domains if xi != xj])

    # Process each arc in the queue
    while queue:
        (xi, xj) = queue.popleft()

        # Revise the domain of xi based on the domain of xj
        if revise(domains, xi, xj):
            # If no valid values are left in xi's domain, return failure
            if not domains[xi]:
                return False  # No solution exists

            # If we revised xi, add arcs (xi, xk) back to the queue for all k ≠ j
            for xk in (domains.keys() - {xi, xj}):
                queue.append((xk, xi))
    
    # Return success if arc-consistency is achieved
    return True

def revise(domains, xi, xj):
    """
    Revise function for AC3
    -----------------------
    Adjusts the domain of xi to ensure that every value in xi has a 
    corresponding value in xj that satisfies the constraints.

    Parameters:
    - domains: A dictionary representing the domains of each variable.
    - xi: The current variable to revise.
    - xj: The other variable in the arc.

    Returns:
    - True if the domain of xi was revised, False otherwise.
    """
    revised = False
    # Iterate over a copy of xi's domain
    for x in set(domains[xi]):
        # If no value in xj's domain satisfies the constraints with x, remove x
        if not any(satisfies_constraints(xi, x, xj, y) for y in domains[xj]):
            domains[xi].remove(x)
            revised = True
    
    return revised

def satisfies_constraints(xi, vx, xj, vy):
    """
    Checks if assigning vx to xi and vy to xj satisfies the problem's constraints.

    Parameters:
    - xi: The first variable.
    - vx: The value for the first variable.
    - xj: The second variable.
    - vy: The value for the second variable.

    Returns:
    - True if the assignment satisfies the constraints, False otherwise.
    """
    # Construct a temporary assignment with placeholder values to match the expected format
    if xi in people:
        temp_assignment = [(xi, vx, "PlaceholderWeapon")]
    elif xi in rooms:
        temp_assignment = [("PlaceholderPerson", xi, vx)]
    else:
        temp_assignment = [("PlaceholderPerson", "PlaceholderRoom", xi)]
    
    if xj in people:
        temp_assignment.append((xj, vy, "PlaceholderWeapon"))
    elif xj in rooms:
        temp_assignment.append(("PlaceholderPerson", xj, vy))
    else:
        temp_assignment.append(("PlaceholderPerson", "PlaceholderRoom", xj))
    
    # Check if the temporary assignment satisfies the constraints
    return is_valid(temp_assignment)

# Preprocess with AC3 to enforce arc-consistency
if ac3(domains):
    print("AC3 succeeded, domains are arc-consistent.")
else:
    print("AC3 failed, no solution exists.")

# Backtracking algorithm
def backtrack(assignment):
    """
    Perform backtracking to solve the puzzle.
    ------------------------------------------
    This function explores possible assignments of variables
    recursively, backtracking when a partial assignment does not
    satisfy the constraints.

    Parameters:
    - assignment: The current partial assignment of variables.

    Returns:
    - A complete assignment that satisfies all constraints if successful, None otherwise.
    """
    # If the assignment is complete (all variables are assigned), return the assignment
    if len(assignment) == 6:
        return assignment
    
    # Iterate through all people (variables)
    for person in people:
        if person not in [p for p, r, w in assignment]:
            # Try each room for the current person
            for room in rooms:
                if room not in [r for p, r, w in assignment]:
                    # Try each weapon for the current person
                    for weapon in weapons:
                        if weapon not in [w for p, r, w in assignment]:
                            # Create a new assignment by adding the current person-room-weapon combination
                            new_assignment = assignment + [(person, room, weapon)]
                            # Check if the new assignment is valid
                            if is_valid(new_assignment):
                                # Recursively attempt to complete the assignment
                                result = backtrack(new_assignment)
                                if result:
                                    return result
    
    # Return None if no valid assignment was found
    return None

# Find the solution using backtracking
solution = backtrack(assignments)

# Print the solution
if solution:
    print("Solution found:")
    for person, room, weapon in solution:
        print(f"{person} was in the {room} with the {weapon}")
    print('Num Checks', num_checks)
else:
    print("No solution found.")

# Reset the number of checks
num_checks = 0

# Function to perform forward checking
def forward_check(domains, person, room, weapon